# Introduction

This notebook introduces the syntax of formulating a model in Gurobi, specifically showing  a variety of different ways to formulate a Gurobi optimization model.  The initial examples are shown for clarity and for the purposes of showing concrete examples of what the Gurobi statements accomplish.  Later examples are more robust for the purposes of constructing large models and also faster than earlier formulations.  Most of the latter formulations will be sufficiently fast for problems of the scale often encountered in real applications.

# A Brief Tour of the <code>gurobipy</code> Package

The first thing we need to do is to import the <code>gurobipy</code> package, which we do using an alias <code>gp</code>, along with importing the `GRB` namespace from that package:

``` python
import gurobipy as gp
from gurobipy import GRB
```
All optimization model components are contained within the <code>gurobipy Model</code> object.  So the first thing we do in a Gurobi optimization model after inputting the data is to establish a variable to represent the model we are building.

``` python
m = gp.Model('name_of_model')
```

The syntax for subsequently adding decision variables, an objective function, and constraints are shown and illustrated below.

The statements used to construct a Gurobi optimization model often use functionality from the <code>gurobipy</code> package, such as a function for faster summing of terms, specifying decision variable types, specifying whether models are to be maximized or minimized, and specifying whether constraints are equality or inequality relationships.

The function for faster summing is, in general,

``` python
gp.quicksum()
```

This function sums the terms within the parentheses, which are often stated using a <code>for</code> statement as we would use in a list comprehension.  If, for example, the variable <code>c</code> was an array of coefficients and <code>x</code> were decision variables constructed in a fashion so that we can iterate through them (in a list perhaps) then we might write something like:

``` python
gp.quicksum(c[i] * x[i] for i in range(len(c)))
```

This example above would be a way to specify an objective function, but <code>gp.quicksum()</code> can be used in a similar way within constraints as well, as we will see in the examples below.

The <code>gurobipy.GRB</code> "namespace" provides a lot of the other functionality, for example,

- Specifying constraint relationships (these can sometimes be specified with ==, <=, and >=)
  - <code>GRB.EQUAL</code>
  - <code>GRB.GREATER_EQUAL</code>
  - <code>GRB.LESS_EQUAL</code>
- Specifying maximization or minimization
  - <code>GRB.MAXIMIZE</code>
  - <code>GRB.MINIMIZE</code>
- Specifying decision variable types
  - <code>GRB.CONTINUOUS</code>
  - <code>GRB.INTEGER</code>
  - <code>GRB.BINARY</code>

# Gurobi Statements to Add Decision Variables

Decision variables can be added to Gurobi models using several statements.  Those statements used in this notebook are listed below and clicking on each bullet will direct you to Gurobi's documentation page for that statement.

- [<code>m.addVar()</code>](https://www.gurobi.com/documentation/current/refman/py_model_addvar.html)
  - Adds a single decision variable
- [<code>m.addVars()</code>](https://www.gurobi.com/documentation/current/refman/py_model_addvars.html)
  - Adds multiple decision variables
- [<code>m.addMVar()</code>](https://www.gurobi.com/documentation/current/refman/py_model_addmvar.html)
  - Add multiple decision variable in a vector/matrix format that behaves like a <code>numpy</code> array in computations

Note also that decision variables can be added directly to the Gurobi model without any intervening Python structure.  Alternately, decision variables can be contained within Python structures such as lists, which might present a convenient way to access the variables to check their value in the final solution.  That said, variables are accessible even if they are not packaged within a Python list.

# Gurobi Statements to Add Constraints

Constraints can be constructed with several Gurobi statements, many of which will be demonstrated in this notebook.  Checking the documentation for a constraint statement can be informative because some permit flexibility in how the constraint data are specified.  For convenience, links to Gurobi documentation for each of these are listed below:

- [<code>m.addLConstr()</code>](https://www.gurobi.com/documentation/current/refman/py_model_addlconstr.html)
  - Adds a single linear constraint
- [<code>m.addConstr()</code>](https://www.gurobi.com/documentation/current/refman/py_model_addconstr.html)
  - Adds a single constraint.  Will sense if the constraint is linear or not.
- [<code>m.addConstrs()</code>](https://www.gurobi.com/documentation/current/refman/py_model_addconstrs.html)
  - Adds multiple constraints and requires a <code>for</code> statement
- [<code>m.addMConstr()</code>](https://www.gurobi.com/documentation/current/refman/py_model_addmconstr.html)
  - Adds a matrix/vector of constraints

# Most Basic Gurobi Model: Hard-Coded Coefficients

This is the simplest and most understandable approach as a first example, although it is infeasible for larger data sets/models.  We nonetheless use this as a starting point for understanding what the more complex approaches do in principal.

Let's revisit our clothing example which had the formulation:

| | | |
| --- | --- | --- |
| Let | | |
| $$s$$ | = | number of shirts to produce next week |
| $$p$$ | = | number of PJs to produce next week |

| | | | | | | |
| --- | --- | --- | --- | --- | --- | --- |
| $\max$ | $4s$ | $+$ | $3p$ | | | |
| s.t. | $1.5s$ | $+$ | $2p$ | $\le$ | $450$ | {cutting hours} |
| | $1s$ | $+$ | $0.75p$ | $\le$ | $280$ | {sewing hours} |
| | $s$ | | | $\ge$ | $0$ | {non-negativity} |
| | | | $p$ | $\ge$ | $0$ | {non-negativity} |

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# create Gurobi model
m = gp.Model('clothing')

# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE  # can be specified when the objective function is defined
m.setParam('TimeLimit', 7200)

# Create decision variables
s = m.addVar(vtype=GRB.CONTINUOUS, name='shirts', lb=0.0)
p = m.addVar(vtype=GRB.CONTINUOUS, name='pajamas', lb=0.0)

# Update model to include variables and parameters
m.update()

# Create constraints
m.addConstr(1.5 * s + 2.0 * p <= 450.0, "CuttingHours")
m.addConstr(1.0 * s + 0.75 * p <= 280.0, "SewingHours")

# Set objective function
m.setObjective(4.0 * s + 3.0 * p, GRB.MAXIMIZE) # redundant model sense specification

# Update the model to incorporate objective and constraints
m.update()

# Optimize the model
m.optimize()

# Most Basic Model with Some Twists

Objective function coefficients can be specified in the decision variable statement.  This is convenient in this case, but might not be convenient with larger models.  Still it is mentioned here to show the latitude that exists in specifying a model.

Also illustrated in the code cell below is an alternate way to specify constraints using < <=, >, >=, and == to indicate the constraint sense as opposed to the <code>GRB</code> namespace sense operators.

In [ ]:
# I have re-imported here simply so that this code cell can be
# run in isolation. In general, it is *bad* practice to re-import.
import gurobipy as gp
from gurobipy import GRB

# create Gurobi model
m = gp.Model('clothing')

# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE  # can be specified when the objective function is defined
m.setParam('TimeLimit', 7200)

# Create decision variables
''' Note that the objective function coefficients are specified here with the "obj" parameter '''
s = m.addVar(vtype=GRB.CONTINUOUS, name='shirts', lb=0.0, obj=4)
p = m.addVar(vtype=GRB.CONTINUOUS, name='pajamas', lb=0.0, obj=3)

# Update model to include variables and parameters
m.update()

# Create constraints
m.addConstr(1.5 * s + 2.0 * p <= 450.0, "CuttingHours")
m.addConstr(1.0 * s + 0.75 * p <= 280.0, "SewingHours")

# Set objective function
''' This statement is now not needed '''
#m.setObjective(4.0 * s + 3.0 * p, GRB.MAXIMIZE) # redundant model sense specification

# Update the model to incorporate objective and constraints
m.update()

# Optimize the model
m.optimize()

# A More Concise & Flexible Formulation for Larger Models

This formulation uses data in Python variables, although they could as easily be in external data files or in a MySQL database.  Its features include:

- Constraints and the objective function are specified with generators (similar to list comprehension) such that the coefficients need not be hard-coded and the formulation syntax will work for a model of any size.
- An alternative for creating a Python list of decision variables using list comprehension is included.
- Decision varables within Python lists provide a convenient way to iterate through the decisions variables as, for example, the objective function is expressed.
- The <code>gp.quicksum()</code>  is a relatively fast way to sum the products of coefficients and decision variables for variables defined in this manner and iterated through using <code>for</code> statements.
- Printouts of sensitivity analysis.  For more information relating to the variable and constraint attributes in regard to sensitivity analysis, please see this URL [Gurobi attributes](https://docs.gurobi.com/projects/optimizer/en/current/reference/attributes.html#secattributereference).

In [ ]:
# Again, re-importing for code cell isolation
import gurobipy as gp
from gurobipy import GRB

''' Data in Python Lists '''
cnstr_coeff = [[1.5, 2.0],[1.0, 0.75]]
cnstr_names = ['CuttingHours', 'SewingHours']
cnstr_rhs = [450.0, 280.0]
obj_coeff = [4.0, 3.0]
var_name = ['shirts', 'pajamas']

# Create Gurobi model
m = gp.Model('clothing')

# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 7200)     

# Create decision variables in a Python list
dvars = []
for i in range(len(obj_coeff)):
    dvars.append(m.addVar(vtype=GRB.CONTINUOUS, name=var_name[i], lb=0.0))
''' Alternate list comprehension method for defining variables '''
#dvars = [m.addVar(vtype=GRB.CONTINUOUS, name=var_name[i], lb=0.0) for i in range(len(obj_coeff))]

# Update model to include variables and parameters
m.update()

# Create constraints in a loop
for i in range(len(cnstr_coeff)):
    m.addLConstr(gp.quicksum((cnstr_coeff[i][j] * dvars[j] for j in range(len(dvars)))),
                 GRB.LESS_EQUAL,
                 rhs=cnstr_rhs[i],
                 name=cnstr_names[i])

# Create objective function
m.setObjective(gp.quicksum(obj_coeff[i] * dvars[i] for i in range(len(dvars))), GRB.MAXIMIZE)

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub}), Reduced cost = {var.RC}, Coeff = {var.obj}, Obj coeff range = ({var.SAObjLow: .3f}, {var.SAObjUp: .3f})')

''' Print sensitivity analysis information on constraints '''
for c in m.getConstrs():
    print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, slack = {c.slack}, Limits = ({c.SARHSLow}, {c.SARHSUp})')

# A Formulation Using Gurobi <code>LinExpr</code>

Formulating objective functions and constraints using <code>LinExpr()</code> is proportedly faster than using <code>quicksum()</code>.  It is also, perhaps, more succinct because one level of <code>for</code> loop or list comprehension are no longer needed.

This formation also uses list comprehension.

[Link to Gurobi Model.LinExpr](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/linexpr.html)

In [ ]:
# Re-importing for code cell isolation
import gurobipy as gp

A = [[1.5, 2.0],[1.0, 0.75]]
cnstr_names = ['CuttingHours', 'SewingHours']
b = [450.0, 280.0]
c = [4.0, 3.0]
var_name = ['q_shirts', 'q_pajamas']

# Create Gurobi model
m = gp.Model('clothing')

# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit', 7200)

# Create decision variables using list comprehension
x = [m.addVar(vtype=GRB.CONTINUOUS, name=var_name[i], lb=0.0) for i in range(len(var_name))]

# Update model to include variables
m.update()

# Create constraints
#for i in range(len(cnstr_rhs)):
#    m.addLConstr(gpy.LinExpr(cnstr_coeff[i], x), gpy.GRB.LESS_EQUAL, cnstr_rhs[i],cnstr_names[i])
cnstr = [m.addLConstr(gp.LinExpr(A[i], x), GRB.LESS_EQUAL, b[i], cnstr_names[i]) for i in range(len(b))]

# Create objective function
m.setObjective(gp.LinExpr(c, x), GRB.MAXIMIZE)

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub}), Reduced cost = {var.RC}, Coeff = {var.obj}, Obj coeff range = ({var.SAObjLow: .3f}, {var.SAObjUp: .3f})')

''' Print sensitivity analysis information on constraints '''
for c in m.getConstrs():
    print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, slack = {c.slack}, Limits = ({c.SARHSLow}, {c.SARHSUp})')

# Python Matrix Mulitplication and <code>numpy</code>

We introduce the Python matrix multiplication operator here, <code>@</code>.  Whereas the <code>*</code> operator multiplies <code>numpy</code> arrays elementwise, the <code>@</code> operator multiplies <code>numpy</code> arrays and matrices in the sense we would think of in linear algebra.

<code>numpy</code> can be used to make Gurobi models more concise.  The models are also, in a sense, simpler, as long as we understand linear algebra and how vector-matrix multiplication is done.

In [ ]:
# import numpy
import numpy as np

# Create a 3 by 3 matrix called A that will
# be filled with numbers 0 through 8
A = np.arange(9).reshape(3,3)

# Create an empty 3 by 3 matrix called B
B = np.empty((3,3))

# Fill all elements of B with 2
B.fill(2)

# print out the two matrices
print(f'A has type {type(A)} and is:\n{A}')
print(f'\nB has type {type(B)} and is:\n{B}')

In [ ]:
# This is NOT matrix multiplication
A * B

In [ ]:
# This IS matrix multiplication
A@B

In [ ]:
# Create a vector called x
x = np.array([1,2,3])

# See what x looks like
x

In [ ]:
# Try matrix multiplication A@x
A@x

In [ ]:
# Try matrix multiplication x@A
x@A

Note that a 1D numpy array is interpreted either as a valid row vector or column vector depending on which side of A it appears in the matrix multiplcation.

When data are of the the <code>numpy matrix</code> data type, then the matrix dimensions must adhere to linear algebra conventions.  The matrix-vector multiplication below will not work because the vector <code>x1</code> does not have three rows to match the number of columns in <code>A1</code>.  Also, as a warning, the <code>matrix</code> data type might be removed from <code>numpy</code> in the future.  So, perhaps best to use <code>numpy</code> arrays.  

In [ ]:
A1 = np.matrix(A)
x1 = np.matrix(x)
A1,x1

In [ ]:
A1@x1

In [ ]:
# Reshape x1 to have 3 rows and 1 column
x1 = x1.reshape(3,1)
x1

In [ ]:
A1@x1

# Formulations Using numpy Arrays

Using <code>numpy</code> arrays for coefficients might be a bit faster and permit objective functions and constraints to be specified using matrix-vector math.  Thus, multiple constraints can be specified in a single statement.

New statements:
- <code>Model.addMVar()</code>
- <code>Model.addMConstr()</code>
- <code>Model.addConstrs()</code>

First, a more concise restatement of the previous formulation using the multiplication of  <code>numpy</code> arrays.  Notice that Gurobi decision variables declared with <code>Model.addMVar()</code> behave like <code>numpy</code> arrays.

The formulation below uses the <code>m.addMVar()</code> method of constructing multiple constraints, which works when the constraint has a particular structure.  Otherwise, you can revert to the <code>m.addConstrs()</code> approach above.

In [ ]:
# Re-importing for code cell isolation
import gurobipy as gp
from gurobipy import  GRB
import numpy as np

A = np.array([[1.5,2.0],[1.0,0.75]])
cnstr_names = ['CuttingHours', 'SewingHours']
b = np.array([450.0,280.0])
c = np.array([4.0,3.0])
var_name = ['shirts', 'pajamas']

# Create Gurobi model
m = gp.Model('clothing')

# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE
m.setParam('TimeLimit',7200)

# Create decision variables using addMVar function
x = m.addMVar(shape=(2), vtype=GRB.CONTINUOUS, name=var_name) 

# Update model to include variables
m.update()

# Create constraints
#cnstr = [m.addLConstr(gp.LinExpr(A[i], x), gp.GRB.LESS_EQUAL, b[i], cnstr_names[i]) for i in range(len(b))]
#cnstr = [m.addConstr(A[i] @ x <= b[i], cnstr_names[i]) for i in range(len(b))]
''' But, this is more succinct '''
#m.addConstrs((A[i] @ x <= b[i] for i in range(len(b))), name=cnstr_names)
''' And, this is even more succinct '''
#m.addConstrs(A @ x <= b for i in range(1))
''' And, maybe even better '''
m.addMConstr(A, x, GRB.LESS_EQUAL, b, name=cnstr_names)

# Create objective function
m.setObjective(c @ x, GRB.MAXIMIZE)

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub}), Reduced cost = {var.RC}, Coeff = {var.obj}, Obj coeff range = ({var.SAObjLow: .3f}, {var.SAObjUp: .3f})')

''' Print sensitivity analysis information on constraints '''
for c in m.getConstrs():
    print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, slack = {c.slack}, Limits = ({c.SARHSLow}, {c.SARHSUp})')

# Reading <code>numpy</code> Model Data from Files

This formulation generates <code>numpy</code> arrays from text files.

In [ ]:
# Re-import for code cell isolation
import gurobipy as gp
from gurobipy import GRB
import numpy as np

A = np.genfromtxt('A.txt')
b = np.genfromtxt('b.txt').reshape(2,1)
c = np.genfromtxt('c.txt')

# Create Gurobi model
m = gp.Model('clothing')

# Specify how to optimize and time limit (seconds)
m.setParam('TimeLimit', 7200)


# Create matrix of decision variables
x = m.addMVar(shape=((2,1)), vtype=GRB.CONTINUOUS, name='x') 

# Update model to include variables
m.update()
print(x)

# Create constraints
m.addConstr(A @ x <= b, name='cnstr') 
''' Alternative statement with m.addMConstr() 
      - Requires a 1D numpy array '''
# m.addMConstr(A, x.reshape((2,)), gp.GRB.LESS_EQUAL, b.reshape((2,)), name='what')

m.update()
for cn in m.getConstrs():
    print(m.getRow(cn), cn.RHS)


# Create objective function
m.setObjective(c @ x, GRB.MAXIMIZE)

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub}), Reduced cost = {var.RC}, Coeff = {var.obj}, Obj coeff range = ({var.SAObjLow: .3f}, {var.SAObjUp: .3f})')

''' Print sensitivity analysis information on constraints '''
for c in m.getConstrs():
    print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, slack = {c.slack}, Limits = ({c.SARHSLow}, {c.SARHSUp})')

**&copy; 2024 - Present: Matthew D. Dean, Ph.D.   
Clinical Professor of Business Analytics at William \& Mary.**